In [ ]:
!pip install python-binance
!pip install tqdm

     |████████████████████████████████| 63 kB 2.5 MB/s 
     |████████████████████████████████| 1.1 MB 33.4 MB/s 
     |████████████████████████████████| 216 kB 66.5 MB/s 
     |████████████████████████████████| 103 kB 62.0 MB/s 
     |████████████████████████████████| 288 kB 63.3 MB/s 
     |████████████████████████████████| 271 kB 71.5 MB/s 
     |████████████████████████████████| 192 kB 54.3 MB/s 
     |████████████████████████████████| 160 kB 54.4 MB/s 


In [ ]:
import numpy as np #pip install numpy
from tqdm import tqdm #pip install tqdm
from binance.client import Client #pip install python-binance
import pandas as pd #pip install pandas
from datetime import datetime
import random

SMA_LOW = 40
SMA_HIGH = 150

def compute_sma(data, window):
    sma = data.rolling(window=window).mean()
    return sma


#select cryptocurrencies you'd like to gather and time interval
ratios = ['BTC']
START_TIME = '7 Nov, 2021'
END_TIME = '21 Nov, 2021'
api_key=''
api_secret=''

client = Client(api_key=api_key,api_secret=api_secret)

merge = False
for ratio in ratios:
    print(f'Gathering {ratio} data...')
    data = client.get_historical_klines(symbol=f'{ratio}USDT',interval=Client.KLINE_INTERVAL_1MINUTE,start_str=START_TIME,end_str=END_TIME)
    cols = ['time','Open','High','Low',f'{ratio}-USD_close',f'{ratio}-USD_volume','CloseTime','QuoteAssetVolume','NumberOfTrades','TBBAV','TBQAV','null']
    
    temp_df = pd.DataFrame(data,columns=cols)
    temp_df = temp_df[['time',f'{ratio}-USD_close']]
    
    if merge == False:
        df = temp_df
    else:
        df = pd.merge(df,temp_df,how='inner',on='time')
    merge = True
    print('complete')
    #sleep for a bit so the binance api doesn't kick you out for too many data asks


for col in df.columns:
    if col != 'time':
        df[col] = df[col].astype(np.float64)

for ratio in ratios:
    df[f'{ratio}_{SMA_LOW}'] = compute_sma(df[f'{ratio}-USD_close'], SMA_LOW)
    df[f'{ratio}_{SMA_HIGH}'] = compute_sma(df[f'{ratio}-USD_close'], SMA_HIGH)
    
#clip NaNs
df = df[SMA_HIGH:]
df = df.reset_index(drop=True)

#convert binance timestamp to datetime
for i in tqdm(range(len(df))):
    df['time'][i] = datetime.fromtimestamp(int(df['time'][i]/1000))

df.to_csv('12-coins-Mar18_Jun20')

Gathering BTC data...
complete


  0%|          | 0/20011 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 20011/20011 [00:03<00:00, 5579.42it/s]


In [ ]:
df


,time,BTC-USD_close,BTC_40,BTC_150
0,2021-11-07 02:30:00,61872.48,61839.10850,61761.637000
1,2021-11-07 02:31:00,61908.31,61838.48350,61764.665200
2,2021-11-07 02:32:00,61892.53,61839.79900,61767.593133
3,2021-11-07 02:33:00,61907.45,61841.24400,61770.746400
4,2021-11-07 02:34:00,61940.13,61843.38475,61774.071400
...,...,...,...,...
20006,2021-11-20 23:56:00,59674.87,59669.54575,59638.593133
20007,2021-11-20 23:57:00,59681.16,59670.14875,59639.683400
20008,2021-11-20 23:58:00,59673.13,59670.73000,59640.816867
20009,2021-11-20 23:59:00,59707.51,59673.16725,59642.247867


In [ ]:
import matplotlib.pyplot as plt

df.plt()

AttributeError: ignored

In [ ]:
class Memory:
  def __init__(self):
    self.clear()

  def clear(self): 
    self.actions = []
        
  def add_to_memory(self, new_action): 
    self.actions.append(new_action)

In [ ]:
DAYS_PER_EPISODE = 1
MINUTES_PER_EPISODE = 1440*DAYS_PER_EPISODE
NUM_EPISODES = 100
TRADING_FEE_MULTIPLIER = .99925 #this is the trading fee on binance VIP level 0 if using BNB to pay fees

class Env:
    def __init__(self, ratios, df):
        self.ratios = ratios
        self.main_df = df
        self.reset()
            
    def reset(self):
        self.balances = {'USD':1.0}
        for ratio in self.ratios:
            self.balances[ratio] = 0.0
        self.iloc = random.randint(0,len(self.main_df)-MINUTES_PER_EPISODE-1)
        self.episode_df = self.main_df[self.iloc:self.iloc+MINUTES_PER_EPISODE+2]
        self.money_in = 'USD'
        
        self.start_time = self.episode_df['time'].iloc[0]
        self.end_time = self.episode_df['time'].iloc[-1]
        
    def step(self):
        self.iloc+=1
        
        #-------IMPLEMENT STRATEGY HERE--------
        if self.money_in == 'USD':
            for ratio in self.ratios:
                
                #if low sma crosses above high sma
                if self.episode_df[f'{ratio}_{SMA_LOW}'][self.iloc] > self.episode_df[f'{ratio}_{SMA_HIGH}'][self.iloc] and self.episode_df[f'{ratio}_{SMA_LOW}'][self.iloc-1] > self.episode_df[f'{ratio}_{SMA_HIGH}'][self.iloc-1]:
                    self.to_buy = ratio
                    #buy that ratio (self.to_buy)
                    self.balances[self.to_buy] = (self.balances['USD']/self.episode_df[f'{self.to_buy}-USD_close'][self.iloc])*TRADING_FEE_MULTIPLIER
                    self.balances['USD'] = 0.0
                    self.buy_price = self.episode_df[f'{self.to_buy}-USD_close'][self.iloc]
                    memory.add_to_memory(f'Buy {self.to_buy}: {self.buy_price}')
                    self.money_in = self.to_buy
                    break
        
        if self.money_in != 'USD': #can't sell if money_in usd
            if self.episode_df[f'{self.money_in}_{SMA_LOW}'][self.iloc] < self.episode_df[f'{self.money_in}_{SMA_HIGH}'][self.iloc]:
                #if high sma crosses below low sma
                #sell money_in/USD
                self.balances['USD'] = (self.balances[self.money_in]*self.episode_df[f'{self.money_in}-USD_close'][self.iloc])*TRADING_FEE_MULTIPLIER
                self.balances[self.money_in] = 0.0
                self.sell_price = self.episode_df[f'{self.money_in}-USD_close'][self.iloc]
                memory.add_to_memory(f'Sell {self.money_in}: {self.sell_price}')
                self.money_in = 'USD'
        #-------IMPLEMENT STRATEGY HERE--------
        
        #-------CALCULATE PERFORMANCE METRICS HERE-------
        #Running net worth
        self.net_worth = self.balances['USD']
        for ratio in self.ratios: 
            self.net_worth += self.balances[ratio]*self.episode_df[f'{ratio}-USD_close'][self.iloc]
        
        #Net_worth had you owned all ratios over episode_df --> 'average_market_change'
        self.average_start_price = 0
        self.average_end_price = 0
        for ratio in self.ratios:
            self.average_start_price += self.episode_df[f'{ratio}-USD_close'].iloc[0]
            self.average_end_price += self.episode_df[f'{ratio}-USD_close'].iloc[-1]
        self.average_start_price /= len(ratios)
        self.average_end_price /= len(ratios)
        self.average_market_change = self.average_start_price / self.average_end_price
        #-------CALCULATE PERFORMANCE METRICS HERE-------
        
        return self.net_worth, self.average_market_change, self.start_time, self.end_time

In [ ]:
df = pd.read_csv('12-coins-Mar18_Jun20')

env = Env(ratios, df)
memory = Memory()

net_worth_collect = []
average_market_change_collect = []

for i_episode in range(NUM_EPISODES):
    for i in range(len(env.episode_df)-1):
        net_worth, average_market_change, start_time, end_time = env.step()
    
    net_worth_collect.append(net_worth)
    average_market_change_collect.append(average_market_change)
    
    #log after each episode
    print(f'episode: {i_episode}')
    print(memory.actions)
    print('\n')
    print(f'interval: {start_time} - {end_time}')
    print(f'net worth after {DAYS_PER_EPISODE} day(s): {net_worth}')
    print(f'average market change: {average_market_change}')
    print('\n')

    memory.clear()
    env.reset()

#log overall
print(f'net worth average after {NUM_EPISODES} backtest episodes: {np.mean(net_worth_collect)}')
#Yes, average of the average market changes
print(f'average, average market change over {NUM_EPISODES} episodes: {np.mean(average_market_change_collect)}')

episode: 0
['Buy BTC: 66840.11', 'Sell BTC: 66771.98', 'Buy BTC: 67598.78', 'Sell BTC: 68434.03', 'Buy BTC: 64696.99', 'Sell BTC: 64549.6', 'Buy BTC: 64614.29', 'Sell BTC: 64541.18', 'Buy BTC: 64689.53', 'Sell BTC: 64648.06', 'Buy BTC: 65123.77']


interval: 2021-11-10 10:51:00 - 2021-11-11 10:52:00
net worth after 1 day(s): 1.0014738606739262
average market change: 1.023449157472285


episode: 1
['Buy BTC: 65119.38', 'Sell BTC: 64789.04', 'Buy BTC: 64812.95', 'Sell BTC: 64626.25', 'Buy BTC: 64929.92', 'Sell BTC: 64743.58', 'Buy BTC: 64391.76', 'Sell BTC: 64401.04', 'Buy BTC: 64613.18', 'Sell BTC: 64350.55', 'Buy BTC: 64309.99']


interval: 2021-11-14 00:22:00 - 2021-11-15 00:23:00
net worth after 1 day(s): 1.003968832758819
average market change: 0.9860876130141165


episode: 2
['Buy BTC: 57721.0', 'Sell BTC: 57748.0', 'Buy BTC: 58250.05', 'Sell BTC: 57856.53', 'Buy BTC: 57039.98', 'Sell BTC: 56805.69', 'Buy BTC: 56128.0', 'Sell BTC: 56155.96', 'Buy BTC: 57148.99', 'Sell BTC: 56841.45